# Photometry Analysis

Welcome to the Neurophotometrics Photometry Analysis Tool! This pipeline will allow you to process your photometry data without the need to write code. The goal of this script is to:

1. Import photometry data and crop
1. Deinterleave data
1. Normalize photometry data for photobleaching and calculate dF/F
1. Segment photometry data around timestamps to create peri-event plots
1. Save figures and processed data

If you are new to the Photometry Analysis Tool, please see our guide here (add link eventually)

For resources and more information on each processing step, see our [Analysis Manual](https://static1.squarespace.com/static/60ff345fca665d50e1adc805/t/645405b5e520f5323850b0fb/1683228085778/Manual-Analysis_20230503.pdf).

## Import Data and Set Parameters


In [ ]:
import os

import pandas as pd
from IPython.display import display
from ipywidgets import Layout, widgets

from routine.plotting import plot_events, plot_signals
from routine.processing import photobleach_correction
from routine.utilities import load_data, pool_events

In [ ]:
exp_layout = Layout(width="auto")
w_data = widgets.FileUpload(
    accept=".csv", multiple=False, description="Upload Data File", layout=exp_layout
)
w_ts = widgets.FileUpload(
    accept=".csv", multiple=False, description="Upload TimeStamp", layout=exp_layout
)
display(w_data)
display(w_ts)

In [ ]:
os.makedirs("./upload", exist_ok=True)
with open("./upload/data.csv", "wb") as dat:
    dat.write(w_data.value[0]["content"].tobytes())
with open("./upload/timestamp.csv", "wb") as dat:
    dat.write(w_ts.value[0]["content"].tobytes())
styling = {"style": {"description_width": "initial"}, "layout": Layout(width="80%")}
data = pd.read_csv("./upload/data.csv")
w_roi = widgets.SelectMultiple(
    options=data.columns, description="ROIs (CTRL click to Select Multiple)", **styling
)
w_nfm = widgets.IntSlider(
    min=0,
    max=data["FrameCounter"].max(),
    step=1,
    description="Number of Frames to Discard from Beginning of Recording",
    **styling
)
w_evt_range = widgets.IntRangeSlider(
    min=-1000,
    max=1000,
    step=1,
    description="Number of Frames to Include Before and After Event ",
    **styling
)
w_base = widgets.ToggleButtons(
    options=["415nm", "470nm", "560nm"],
    description="Channel to use as Reference Signal:",
    disabled=False,
    button_style="",
    tooltips=[
        "Best for most recordings",
        "Alternative for certain neurotransmitter sensors",
        "Alternative for certain red-shifted sensors",
    ],
)

display(w_roi)
display(w_nfm)
display(w_evt_range)
display(w_base)

In [ ]:
rois = list(w_roi.value)
w_roi_dict = {
    r: widgets.Text(value=r, description="Label for {}".format(r), **styling)
    for r in rois
}
for w in w_roi_dict.values():
    display(w)

In [ ]:
IN_DATA = "./upload/data.csv"
IN_TS = "./upload/timestamp.csv"
FIG_PATH = "./figs/d0"
OUT_PATH = "./output/d0"
PARAM_NFM_DISCARD = w_nfm.value  # 100
PARAM_EVT_RANGE = w_evt_range.value  # (-200, 400)
PARAM_LED_DICT = {7: "initial", 1: "415nm", 2: "470nm", 4: "560nm"}
PARAM_ROI_DICT = {r: w.value for r, w in w_roi_dict.items()}
PARAM_BASE_SIG = w_base.value  # "415nm"
assert (
    PARAM_BASE_SIG in PARAM_LED_DICT.values()
), "cannot use {} as baseline signal as it's not in LED label mapping: {}".format(
    PARAM_BASE_SIG, PARAM_LED_DICT
)
os.makedirs(FIG_PATH, exist_ok=True)
os.makedirs(OUT_PATH, exist_ok=True)

## Load Data


In [ ]:
data, ts = load_data(IN_DATA, IN_TS, PARAM_NFM_DISCARD, PARAM_LED_DICT, PARAM_ROI_DICT)

## Visualize Raw Signal
Run cell to generate raw data plots


In [ ]:
fig = plot_signals(data, list(PARAM_ROI_DICT.values()))
fig.write_html(os.path.join(FIG_PATH, "raw_signals.html"))
fig.show()

## Correct for Photobleaching


Order of Operations:

1. Deinterleave data by frame flag (LED) and save into a matrix
1. Fit isosbestic (or 470) signal with a biexponential decay 
1. Linearly scale the fitted decay to the 470 data using robust fit.
1. Divide the raw 470 data by this scaled fit to get a corrected signal

For more information on dF/F values, see our [Analysis Manual](https://static1.squarespace.com/static/60ff345fca665d50e1adc805/t/645405b5e520f5323850b0fb/1683228085778/Manual-Analysis_20230503.pdf).



In [ ]:
data_norm = photobleach_correction(data, list(PARAM_ROI_DICT.values()), PARAM_BASE_SIG)

## Visualize Corrected Data


In [ ]:
fig = plot_signals(
    data_norm,
    list(PARAM_ROI_DICT.values()),
    group_dict={
        "415nm": "415nm",
        "415nm-fit": "415nm",
        "470nm": "470nm",
        "470nm-norm": "470nm",
    },
)
fig.write_html(os.path.join(FIG_PATH, "photobleaching_correction.html"))
fig.show()

## Segment Data with `'Keydown'` Timestamps


In [ ]:
evt_df = pool_events(
    ts, data, PARAM_EVT_RANGE, list(PARAM_ROI_DICT.values()), event_name="Key"
)

## Visualize Peri-Event Plots


In [ ]:
fig = plot_events(evt_df, list(PARAM_ROI_DICT.values())[0])
fig.write_html(os.path.join(FIG_PATH, "events.html"))
fig.show()

## Save Processed Data


In [ ]:
data_norm.to_csv(os.path.join(OUT_PATH, "data_norm.csv"), index=False)
evt_df.to_csv(os.path.join(OUT_PATH, "events.csv"), index=False)